<a href="https://colab.research.google.com/github/eelcofolkertsma/mydcc/blob/main/pythonwallet-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NL Wallet App Simulator
- Execute each cell with Crtl+Enter
- Two cells will ask for an input. Make sure the respective QR (init, DCC) is readily a hand for cut and paste


Install libraries if you run this notebook for the first time

In [1]:
!pip install pyjwt cryptography protobuf pycryptodome ecdsa


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 2.0 MB 33.1 MB/s 
     |████████████████████████████████| 119 kB 61.4 MB/s 


### Ingest an invitation

Identify yourself at an airline (e.g. https://pinggg.mywire.org/static-v2) to obtain an invitation for validation. 

Grab the invitation string from url to wallet and paste in input-box. <br>
Or alternative: Open QR-code in Google-lens and "copy text" to capture the invitation string.

### Init QR code handling by wallet app
The wallet app processes the invitation:

1.   The wallet gets airline identity document to learn airline endpoints and validation service location
2.   The wallet determines location of validation service ((from airline identity document)). A choice is presented in case multiple validation services are offered.

In [5]:
import requests
import json
import jwt
import base64

invite = input('Paste readable Invitation string contents here: ')
#skip base64 decompress to avoid issue with TV airline stub 
base64_message = invite
base64_bytes = base64_message.encode('utf-8')
message_bytes = base64.b64decode(base64_bytes)
qr_code = message_bytes.decode('utf-8')
#qr_code = invite
qr_code_data =  json.loads(qr_code)
print(f"QR-Code data: {json.dumps(qr_code_data, indent=4)}")

token_info = jwt.decode(qr_code_data['token'], options={"verify_signature":False})
print(f"Invitation-token: {token_info}")

import ecdsa
import base64
from Crypto.Hash import SHA256
from ecdsa.curves import NIST256p

# Load the airline identity document 
serviceIdentity = requests.get( qr_code_data['serviceIdentity'] ).json()

# Get the information from the identity document. Identity contents
## services, i.e. airline endpoint to get validation access token and to return confirmation token, and (list of) providers that offer validation
## verification methods, public keys to validate signatures or for use in encryption 




Paste readable Invitation string contents here: eyJwcm90b2NvbCI6IkRDQ1ZBTElEQVRJT04iLCJwcm90b2NvbFZlcnNpb24iOiIxLjQuMCIsInNlcnZpY2VJZGVudGl0eSI6Imh0dHBzOi8vcGluZ2dnLm15d2lyZS5vcmcvd2FsbGV0L2lkZW50aXR5L3YyIiwicHJpdmFjeVVybCI6Imh0dHBzOi8vc29tZS5pbnN0aXR1dGlvbi92YWxpZGF0aW9uL3ByaXZhY3kiLCJ0b2tlbiI6ImV5SmhiR2NpT2lKRlV6STFOaUlzSW5SNWNDSTZJa3BYVkNJc0ltdHBaQ0k2SWtGcFoyOU9iMlE1Y1VsU2VYRkRXRVJSYjBaNVpXeE5SRTF2V0daMlJtUlVVVXRCVVhGQ1VqUnlaMEVpZlEuZXlKcGMzTWlPaUpvZEhSd2N6b3ZMM0JwYm1kblp5NXRlWGRwY21VdWIzSm5MM2RoYkd4bGRDOXBaR1Z1ZEdsMGVTOTJNaUlzSW5OMVlpSTZJamM1TW1Oa1ptTmxZakU0WmpSaVpqZzRabU0yWmprd09XSXpaRGhpTVdNeElpd2lhV0YwSWpveE5qVTFOREV6TURFNExDSmxlSEFpT2pFMk5UVTBNVFkyTVRoOS5EM3pWS3JUVXpwWHJzdV9DVzJJTzBGSFJuVVZDNU02U3JHUGVVSm1melFZQm14N3NKWU1HVVdxNU8wMmdBT0t0eUNmazRlcm5DcnNkazFvNHZPX0g4QSIsImNvbnNlbnQiOiJQbGVhc2UgY29uZmlybSB0byBzdGFydCB0aGUgRENDIEV4Y2hhbmdlIGZsb3cuIElmIHlvdSBub3QgY29uZmlybSwgdGhlIGZsb3cgaXMgYWJvcnRlZC4iLCJzdWJqZWN0IjoiNzkyY2RmY2ViMThmNGJmODhmYzZmOTA5YjNkOGIxYzEiLCJzZXJ2aWNlUHJvdmlk

#Wallet gets the validation access token
which packs travel data and some instructions for type of validation that is expected. 
- The endpoint to get access token is determined from the airline identity document
- The request is authenticated with invitation-token (so links back to booking and passenger)

Under EU-DCC-validation protocol (but not implemented by us)
- The wallet advises its one-time identity, not sure why??
- The airline hands back validation access token and a nonce (nonce, why??)

In [6]:
import ecdsa
import base64
from Crypto.Hash import SHA256
from ecdsa.curves import NIST256p

#create one-time wallet identity(key-pair) to sign the DCC on upload
userkey = ecdsa.SigningKey.generate(curve=NIST256p,hashfunc=SHA256.new) 
#print(userkey.get_verifying_key().to_der())



pubkey = userkey.get_verifying_key().to_pem().decode()
print(pubkey)
pubkey = ''.join(pubkey.split('\n')[1:-2]) # remove -----BEGIN PUB KEY... lines and concatenate

body={
    "pubKey": pubkey,
    "alg": "SHA256withECDSA"
    }
print(body)

# App selects the AccessTokenService in Airline Identity document and requests for Validation Access token. 

for service in serviceIdentity['service']:
    if service['id'].endswith('#AccessTokenService'):
        response = requests.post( service['serviceEndpoint'], 
                    headers={'Authorization': f'Bearer {qr_code_data["token"]}',
                             'Content-Type':'application/json', 
                             'X-Version' : '2.00'
                             },
                    json = body                               
        )
       
        # This is the access token for the validation service
        validator_token = response.text 
        validator_nonce= base64.b64decode(response.headers['X-Nonce'])
        validator_encKey= json.loads(base64.b64decode(response.headers['X-Enc']).decode())
        validator_sigKey= json.loads(base64.b64decode(response.headers['X-Sig']).decode())
        print(response.text)
        print(validator_nonce)

        token_info = jwt.decode(validator_token, options={"verify_signature":False})
        print(json.dumps(token_info, indent=4))
        print(token_info['aud'])





-----BEGIN PUBLIC KEY-----
MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEX23bBJxcwmU5K0PO8XDG4/qwPpih
ypfuzAb2KwXSMtHW9Qud0PCH3EHcH9/GUYlpKmn4JwcYLkXSVzvRtwxMfg==
-----END PUBLIC KEY-----

{'pubKey': 'MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEX23bBJxcwmU5K0PO8XDG4/qwPpihypfuzAb2KwXSMtHW9Qud0PCH3EHcH9/GUYlpKmn4JwcYLkXSVzvRtwxMfg==', 'alg': 'SHA256withECDSA'}
eyJhbGciOiJFUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IkFpZ29Ob2Q5cUlSeXFDWERRb0Z5ZWxNRE1vWGZ2RmRUUUtBUXFCUjRyZ0EifQ.eyJpc3MiOiJodHRwczovL3BpbmdnZy5teXdpcmUub3JnL3dhbGxldC9pZGVudGl0eS92MiIsInN1YiI6Ijc5MmNkZmNlYjE4ZjRiZjg4ZmM2ZjkwOWIzZDhiMWMxIiwiYXVkIjoiaHR0cHM6Ly90dnZhbGlkYXRpb24ubXl3aXJlLm9yZy92YWxpZGF0ZS83OTJjZGZjZWIxOGY0YmY4OGZjNmY5MDliM2Q4YjFjMSIsInQiOjIsInYiOiIxLjAiLCJ2YyI6eyJsYW5nIjoiZW4tZW4iLCJjb2EiOiJOTCIsImNvZCI6IkJFIiwicG9hIjoiQU1TIiwicG9kIjoiQlJVIiwicm9hIjoiTkwiLCJyb2QiOiJCRSIsInR5cGUiOlsiciIsInYiLCJ0Il0sImNhdGVnb3J5IjpbIlN0YW5kYXJkIl0sInZhbGlkYXRpb25DbG9jayI6IjIwMjItMDYtMTZUMjA6NTc6MzguMzMzWiIsInZhbGlkRnJvbSI6IjIwMjItMDYtMTdUMjA6NTY6NTguMjcyWiIsI

# Submit the health certificate
You can enter the health certificate data into an input field. 

Source for DCC: https://eu-dcc-validation.web.app/

Open the QR-code with Google lens (right click option in Chrome) and "copy text" to obtain health sertificate in string format 

Make sure that the name and date-of-birth match the data in the validation access token from above. 

##The wallet app now executes the following steps: 
- choose a random password (32 bytes)
- AES-encrypt the DCC with the password from above and the nonce that was obtained together with the access token 
- encrypt the password with the validation service's public key 
- sign the encrypted AES-encrypted data with the userkey that has been submitted to get the access token
- send JSON data to the validation service:
   - kid of validation server's public key that was used to encrypt password
   - encrypted dcc data
   - signature of encrypted dcc data
   - PKI encrypted password for dcc
   - constants: encScheme = RSAOAEPWithSHA256AESCBC, sigAlg = SHA256withECDSA
   - header: Access token that was previously obtained
- decode the response and print it
  - Private part of result is feedback on evaluation of individual busness rules
  - Public part of result is confirmaion token, which can be made availabe to airline if passenger wishes to do so



In [11]:
hcert = input('Please paste your HCert, or enter "bob": ').strip()  # Option 1: Enter the HC1:-String into the input field

if hcert=='bob':
  hcert= 'HC1:NCF520B90T9WTWGVLK-49NJ3B0J$OCC*AX*4FBB%91*70J+9FN0ZQC%PQWY04GC+2KD97TK0F90$PC5$CUZC$$5Y$5JPCT3E5JDLA73467463W5/A6..DX%DZJC2/D.H8B%E5$CLPCG/DX-CE1AL1BUIAI3DUOCT3EMPCG/DUOC+0AKECTHG4KCD3DX47B46IL6646H*6Z/E5JD%96IA74R6646307Q$D.UDRYA 96NF6L/5SW6Y57B$D% D3IA4W5646946846.96XJC%+D3KC/SCXJCCWENF6OF64W5KF6946WJCT3EJ+9%JC+QENQ4ZED+EDKWE3EFX3ET34X C:VDG7D82BUVDGECDZCCECRTCUOA04E4WEOPCN8FHZA1+92ZAQB9746VG7TS9FTA1N8I%63:6SM87N8 L6T0AUM8OZA.Q6.K427B.MAA2J9MPDYM+KARBW.LSU39L79-2D15RECO1EQMFP10N7CPWGI6LLPEHMHFN/R1IEV*MR1F%6RPNE07SU$J:/7J:O238.W3V50U50QEWMQDR4'
#assert hcert.startswith('HC1:')

from base64 import b64decode
from random import randint
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from ecdsa.util import sigencode_der

# Use real random numbers for production instead!
password = bytearray([randint(0,255) for i in range(32)])

pubkey = validator_sigKey["x5c"][0]

keyDER = b64decode(pubkey)
keyPub = RSA.importKey(keyDER)

validatorkey = keyPub
aesCipher = AES.new(password, AES.MODE_CBC,iv=b64decode(validator_nonce))

cipher = PKCS1_OAEP.new(validatorkey,hashAlgo=SHA256)
cryptKey = cipher.encrypt(password)
ciphertext= aesCipher.encrypt(pad(bytes(hcert,'utf-8'),AES.block_size))
signature = userkey.sign(ciphertext,hashfunc=SHA256.new,sigencode=sigencode_der)

####### validate   ######
headers = {'content-type': 'application/json',
           'accept': 'application/json',
           'X-Version': '2.00',
           'Content-Type': 'application/json',
           "Authorization":"Bearer " + validator_token }

body = {"kid":validator_sigKey["kid"],
        "dcc":base64.b64encode(ciphertext).decode(),
        "sig":base64.b64encode(signature).decode(),
        "encKey":base64.b64encode(cryptKey).decode(),
        "encScheme":"RSAOAEPWithSHA256AESCBC", 
        "sigAlg":"SHA256withECDSA"}

response = requests.post(token_info['aud'], data=json.dumps(body), headers=headers)
print ("Validate Response", response.status_code, response.text, '\n\n')


if response.ok:
  validate_result = jwt.decode(response.content, options={"verify_signature":False})
  print(f'Validate result message: {json.dumps(validate_result, indent=4)}')

Please paste your HCert, or enter "bob": bob
NotdWikdAbpdoI=
Validate Response 400 DCC was either not present, not decrypted correctly or failed sig check. 




#completion, push confirmation
Wallet offers to push confirmation to airline, as final step. Under consent.

On "yes", wallet identifies endpoint from airline identity document and POSTs the confirmation token under authentication with invitation token. In final message traeler is advised to go back to airline site

In [ ]:
ahum = input('Enter yes to push token to arline: ').strip()
  # App selects the ConfirmationService in Airline Identity document and posts the confirmation token.
for service in serviceIdentity['service']:
    if service['id'].endswith('#ConfirmationService-1'):
      response = requests.post( service['serviceEndpoint'],
                               headers={'Authorization': f'Bearer {qr_code_data["token"]}', #initR-token goes here
                               'Content-Type':'application/json', 'X-Version' : '1.0'},
                               json = {"confirmation": validate_result["confirmation"]})
print ("Undecoded Response", response.status_code, response.text, '\n\n')
        
print('Please navigate back to airline website')